In [1]:
import os
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import sys

In [2]:
tsp_dir = os.getcwd() + os.sep + 'TSPLIB' + os.sep

In [3]:
tsp_files_list = os.listdir(tsp_dir)

In [4]:
subprocess.run('make christofides', shell=True, capture_output=True)

CompletedProcess(args='make christofides', returncode=0, stdout=b"make: 'christofides' is up to date.\n", stderr=b'')

In [5]:
out = subprocess.run(['./christofides -f TSPLIB/gr17.tsp'], shell=True, capture_output=True)
output = out.stdout.decode('utf-8')
print(output)
import sys

OptimalSoln:	2085.000000
Simple:		1.053717	2197
Tri-Opt:	1.031655	2151
Tri-Comp:	1.031655	2151
Comp-Heur:	1.074820	2241



In [6]:
def parseOutputString(outputStr):
    lines = outputStr.split('\n')
    for l in lines:
        s = l.split()
        print(s)
    return

In [7]:
parseOutputString(output)

['OptimalSoln:', '2085.000000']
['Simple:', '1.053717', '2197']
['Tri-Opt:', '1.031655', '2151']
['Tri-Comp:', '1.031655', '2151']
['Comp-Heur:', '1.074820', '2241']
[]


In [8]:
def getSupportedFilesList():
    tsp_files_list = os.listdir(tsp_dir)
    supportedFiles = []
    keyWord = 'EDGE_WEIGHT_SECTION'
    for filename in tsp_files_list:
        foundKeyWord = False
        file = open(tsp_dir + os.sep + filename, 'r')
        lines = file.readlines()
        for line in lines:
            if(line.find(keyWord) != -1):
                foundKeyWord = True
                break
        if(foundKeyWord == False):
            supportedFiles.append(filename)
    return supportedFiles

In [9]:
supported_tsp_files_list = getSupportedFilesList()